In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 

### Importing data

In [ ]:
train_df=pd.read_csv("../input/cardiovascular-disease-identification-2nd/train.csv")
test_df=pd.read_csv("../input/cardiovascular-disease-identification-2nd/test.csv")

In [ ]:
train_df.head()

In [ ]:
with open('/kaggle/input/cardiovascular-disease-identification-2nd/data/data/train/C20/0008380') as f:
    lines = f.readlines()
    

In [ ]:
train_path = "../input/cardiovascular-disease-identification-2nd/data/data/train"
test_path="../input/cardiovascular-disease-identification-2nd/data/data/test"
import glob
def list_of_files(path):
    text_files = glob.glob(path + "/*/*", recursive = True)
    return text_files



In [ ]:
import ntpath
def extract_text_from_files(l_files):
    text={"filename":[],"text":[]}
    for file in l_files:
        text["filename"].append(ntpath.basename(file))
        with open(file) as f:
            lines = f.readlines()
        text["text"].append(lines)
    return(text)


In [ ]:
text_train_files=list_of_files(train_path)
l_train=extract_text_from_files(text_train_files)
text_train=pd.DataFrame(l_train)
text_train.filename=text_train.filename.astype(int)
text_train.drop_duplicates(subset="filename",inplace=True)
text_train.sort_values(by="filename")
text_train["text"]=text_train["text"].apply(lambda x : ''.join(x))

In [ ]:
text_train.head()

In [ ]:
train_df=train_df.sort_values(by="filename")

In [ ]:
test_path="../input/cardiovascular-disease-identification-2nd/data/data/test"
text_test_files = glob.glob(test_path + "/*", recursive = True)

In [ ]:
l_test=extract_text_from_files(text_test_files)
text_test=pd.DataFrame(l_test)
text_test.filename=text_test.filename.astype(int)
text_test.drop_duplicates(subset="filename",inplace=True)
text_test.sort_values(by="filename")
text_test["text"]=text_test["text"].apply(lambda x : ''.join(x))

In [ ]:
text_test=text_test.sort_values(by="filename")

In [ ]:
test_df=test_df.sort_values(by="filename")

In [ ]:
X=text_train["text"]
y=train_df.loc[:, train_df.columns != 'filename']

# Preprocessing text

In [ ]:
import re
def clean_text(text):
    #transform in lowercase
    text=text.lower()
    #suppress whitespaces in the begining and the end 
    text=re.sub(r"^\s+|\s+$", "",text )
    #supress caracters that are neither alphabet neither whitespace
    text=re.sub('[^A-Za-z ]+', '', text)
    return text

### lemmatizing text

In [ ]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_text(text):
    tokens = nltk.word_tokenize (text)
    text =' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return text

### remove stopwords

In [ ]:
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

In [ ]:
def strip_stopwords(text, is_lower_case=False):
    tokens = nltk.word_tokenize (text)
    #tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

### Defining cleaning function that preprocess our text

In [ ]:
def cleaning(liste):
    
    processed_text=[]
    for  i in range (len(liste)):
        text=liste[i]
        
        #supprimer les chiffres et supprimer les carctères qui ne sont ni des alphabets ni des espaces ,et les espaces au début et à la fin ,transformer en minuscule,
        text=clean_text(text)
            
        # supprimer extra newlines
        text = re.sub(r'[\r|\n|\r\n]+', ' ',text)
        
        # raciner le texte
        text= lemmatize_text(text)
    
        # supprimer extra espaces
        text= re.sub(' +', ' ', text)
        
        # supprimer les mots vides
        text = strip_stopwords(text, is_lower_case=True)
            
        processed_text.append(text)
        
    return processed_text

In [ ]:
l=list(X)
corpus=cleaning(l)

### Creating matrix document

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(corpus)

### Modeling

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
LR = LogisticRegression(solver='liblinear',random_state=0)
sgd=SGDClassifier()
svc=SVC()

#### spliting data 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.metrics import hamming_loss

In [ ]:
for classifier in [LR,sgd,svc]:
    clf=OneVsRestClassifier(classifier)
    clf.fit(X_train,np.array(y_train))
    y_predict=clf.predict(X_test)
    hamming_accuracy=hamming_loss(y_test,y_predict)
    print("Clf: ",classifier.__class__.__name__)
    print("hamming loss :",hamming_accuracy )

In [ ]:
test_df.head()

we see clearly that svm work better than other classifiers so we will prediction with svm

In [ ]:
clf=OneVsRestClassifier(svc)
clf.fit(X_train,np.array(y_train))
y_predict=clf.predict(X_test)
hamming_accuracy=hamming_loss(y_test,y_predict)
print("Clf: ",svc.__class__.__name__)
print("hamming loss :",hamming_accuracy )

In [ ]:
X_test1=cleaning(list(text_test["text"]))

In [ ]:
X_test1=vectorizer.fit_transform(X_test1)

In [ ]:
predictions=clf.predict(X_test1)

In [ ]:
train_df.columns

In [ ]:
columns=['target_00', 'target_01', 'target_02', 'target_03',
       'target_04', 'target_05', 'target_06', 'target_07', 'target_08',
       'target_09', 'target_10', 'target_11', 'target_12', 'target_13',
       'target_14', 'target_15', 'target_16', 'target_17', 'target_18',
       'target_19', 'target_20', 'target_21', 'target_22']

In [ ]:
test_df[columns]=predictions

In [ ]:
test_df.to_csv("./cardio_submission2.csv",index=False)